In [1]:
from pathlib import Path

BASE_DIR = Path().resolve().parent
print(BASE_DIR)

import pandas as pd
csv_path = BASE_DIR / r'data\raw\combined_news_data.csv'

C:\Users\atalb\Documents\Coding\MLandAI\Fake-news-detection


In [2]:
news_data = pd.read_csv(csv_path)
news_data

,full_text,subject,label
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,News,0
1,Failed GOP Candidates Remembered In Hilarious...,News,0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,News,0
3,California AG pledges to defend birth control ...,politicsNews,1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,politics,0
...,...,...,...
44893,Nigeria says U.S. agrees delayed $593 million ...,worldnews,1
44894,Boiler Room #62 – Fatal Illusions Tune in to t...,Middle-east,0
44895,ATHEISTS SUE GOVERNOR OF TEXAS Over Display on...,Government News,0
44896,Republican tax plan would deal financial hit t...,politicsNews,1


In [3]:
news_data.shape

(44898, 3)

In [4]:
news_data.isna()

,full_text,subject,label
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
44893,False,False,False
44894,False,False,False
44895,False,False,False
44896,False,False,False


In [5]:
news_data['full_text']

0         BREAKING: GOP Chairman Grassley Has Had Enoug...
1         Failed GOP Candidates Remembered In Hilarious...
2         Mike Pence’s New DC Neighbors Are HILARIOUSLY...
3        California AG pledges to defend birth control ...
4        AZ RANCHERS Living On US-Mexico Border Destroy...
                               ...                        
44893    Nigeria says U.S. agrees delayed $593 million ...
44894    Boiler Room #62 – Fatal Illusions Tune in to t...
44895    ATHEISTS SUE GOVERNOR OF TEXAS Over Display on...
44896    Republican tax plan would deal financial hit t...
44897    U.N. refugee commissioner says Australia must ...
Name: full_text, Length: 44898, dtype: object

In [10]:
import re 
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()


def clean_text(text, lemmatize=False):
    text = text.lower()  

    text = remove_html(text)  
    text = re.sub(r"http\S+|www\S+", "", text)   
    text = re.sub(r"[^a-zA-Z]", " ", text)  

    tokens = text.split()   
    tokens = [w for w in tokens if w not in stop_words]  

    # if lemmatize:
    #     tokens = [lemmatizer.lemmatize(w) for w in tokens]  # ✅ optional lemmatization  

    return " ".join(tokens)

In [11]:
clean_df = news_data['full_text'].apply(lambda x: clean_text(x))
clean_df

C:\Users\atalb\AppData\Local\Temp\ipykernel_30072\1049205147.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


0        breaking gop chairman grassley enough demands ...
1        failed gop candidates remembered hilarious moc...
2        mike pence new dc neighbors hilariously trolli...
3        california ag pledges defend birth control ins...
4        az ranchers living us mexico border destroy na...
                               ...                        
44893    nigeria says u agrees delayed million fighter ...
44894    boiler room fatal illusions tune alternate cur...
44895    atheists sue governor texas display capitol gr...
44896    republican tax plan would deal financial hit u...
44897    u n refugee commissioner says australia must s...
Name: full_text, Length: 44898, dtype: object